In [15]:
import pandas as pd
import os
import numpy as np
import codecs

In [17]:
config_path = '/openbayes/input/input0/BERT/cased_L-12_H-768_A-12/bert_config.json'
checkpoint_path = '/openbayes/input/input0/BERT/cased_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '/openbayes/input/input0/BERT/cased_L-12_H-768_A-12/vocab.txt'

maxlen = 200
CLASS_NUM = 3
batch_size = 32

In [ ]:
#编写加载数据的函数
def load_data(path):
    print("[INFO] loading images...")
    data = []
    labels = []
    # grab the image paths and randomly shuffle them
    imagePaths = sorted(list(paths.list_images(path)))
    random.seed(42)
    random.shuffle(imagePaths)
    # loop over the input images
    for imagePath in imagePaths:
        img = image.load_img(imagePath, target_size=(224, 224))
        x = image.img_to_array(img)
        data.append(x)
        #label是上级目录名
        label = int(imagePath.split(os.path.sep)[-2])       
        labels.append(label)
    # scale the raw pixel intensities to the range [0, 1]
    data = np.array(data, dtype="float") / 255.0
    labels = np.array(labels)

    # convert the labels from integers to vectors
    labels = to_categorical(labels, num_classes=CLASS_NUM)                         
    return data,labels

In [9]:
import os
#获取访问图片的路径的list
#输入是图片的上级目录和图片的名字，image_names是从csv文件读出来的list
def get_image_paths(image_path, image_names):
    image_paths = []
    for Image_name in Image_names:
        image_paths.append(os.path.join(image_path,Image_name))
    return image_paths

In [13]:
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([
        np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X
    ])

In [16]:
token_dict = {}

with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)


class OurTokenizer(Tokenizer):
    def _tokenize(self, text):
        R = []
        for c in text:
            if c in self._token_dict:
                R.append(c)
            elif self._is_space(c):
                R.append('[unused1]') # space类用未经训练的[unused1]表示
            else:
                R.append('[UNK]') # 剩余的字符是[UNK]
        return R

tokenizer = OurTokenizer(token_dict)

NameError: name 'dict_path' is not defined

In [20]:
def load_data(text_path, image_path):
    data = pd.read_csv(text_path)
    idxs = list(range(len(data)))
    np.random.shuffle(idxs)
    X1, X2, X3, Y = [], [], [], []
    for i in idxs:
        d = data[i]
        text = d[1][:maxlen]#text暂时先用文字，随后加图片名字上去
        x1, x2 = tokenizer.encode(first=text)
        imagePath = get_image_paths(image_path,d[2])
        img = image.load_img(imagePath, target_size=(224, 224))
        x3 = image.img_to_array(img)
        y = d[0]
        X1.append(x1)
        X2.append(x2)
        X3.append(x3)
        Y.append(y)
        X3 = np.array(X3, dtype="float") / 255.0
        Y = np.array(Y)
        X1 = seq_padding(X1)
        X2 = seq_padding(X2)
        Y = to_categorical(Y, num_classes=CLASS_NUM)
        return [X1, X2, X3], Y
#         if len(X1) == batch_size or i == idxs[-1]:
#             X1 = seq_padding(X1)
#             X2 = seq_padding(X2)
#             Y = seq_padding(Y)
#             return [X1, X2, X3], Y
#             [X1, X2, X3, Y] = [], [], [], []

In [18]:
def square():
    for x in range(4):
        yield x ** 2
square_gen = square()
for x in square_gen:
    print(x)

0
1
4
9


In [19]:
square_gen

<generator object square at 0x0000021E836AEB48>